## Introdução

Formar uma família e ter filhos é um objetivo de vida para muitas pessoas, mas a chegada de um recém-nascido traz mudanças profundas e desafios logísticos imediatos para a casa. Esse cenário é ainda mais desafiador hoje em dia, pois a maioria das mulheres precisa conciliar a maternidade com a carreira profissional, não estando mais disponíveis em tempo integral apenas para o lar.

Essa nova dinâmica exige uma rede de apoio sólida. Como apontado por Patias, Siqueira e Dias (2021), existe uma diferença marcante na rotina de mães que trabalham fora em comparação às que se dedicam exclusivamente à casa, o que torna a divisão justa de tarefas algo essencial para o bem-estar familiar.

Pensando nessa necessidade de organização, este trabalho propõe um modelo computacional para gerenciar as tarefas da casa e do bebê. A ideia é criar uma agenda otimizada que distribua as atividades entre a rede de apoio (pais, familiares, babás), respeitando os horários livres, a carga horária e a habilidade de cada pessoa para realizar determinadas funções.

## Modelo matemático


Para o modelo, será usada Programação Linear Inteira Mista (MILP). 

**Condições estabelecidas**

- Slots fixos de tempo (quantização)
- Tarefas com duração variável (múltiplas de slot)
- Vínculos entre tarefas
- Capacidade e disponibilidade das pessoas
- Cada pessoa realiza uma tarefa por vez 
- Separação de tarefas de casa e do bebê 
- Nao sobreposição de tarefas do bebê
- Horários que as tarefas podem ser feitas

**Conjuntos e Parâmetros**

- Conjunto de Slots (Intervalo de tempo): $\mathcal{T} = \{0 |  1 |  \dots, S-1\}$, com $t \in \mathcal{T}$.
- Conjunto de Pessoas: $\mathcal{P}$, com $i \in \mathcal{P}$.
- Conjunto de Tarefas (Total): $\mathcal{D}$
- Subconjunto de Tarefas do Bebê: $\mathcal{D}_{B} \subset \mathcal{D}$.
- Subconjunto de Tarefas da Casa: $\mathcal{D}_{C} \subset \mathcal{D}$.
(Onde $\mathcal{D}_{B} \cup \mathcal{D}_{C} = \mathcal{D}$ e $\mathcal{D}_{B} \cap \mathcal{D}_{C} = \emptyset$)
- Ocorrências: $\mathcal{O}_j$ para a tarefa $j \in \mathcal{D}$, cada tipo de tarefa $j$ pode ocorrer várias vezes por dia.
- Capacidade (Aptidão): $c_{i,j} \in [0, 1]$, quão bem a pessoa $i$ executa a tarefa $j$.
- Duração: $d_j$ (em slots) para a tarefa $j$.
- Disponibilidade da Pessoa: $PA_{i,t} \in \{0, 1\}$ slot $t$ que a pessoa $i$ está disponível.
- Disponibilidade da Tarefa: $TA_{j,t} \in \{0, 1\}$, indica se a tarefa $j$ pode ser iniciada no slot $t$.
- Limite de Carga: $L_i$, número máximo de slots que a pessoa pode ser alocada para tarefas.
- Peso do Balanceamento: $\alpha$, constante que define a penalidade para o desequilíbrio de carga (ex: 0, para considerar sem penalidade e 1000 para considerar muita penalidade).


**Variável de Decisão**

Variável binária principal:

$$x_{i,j,o,t} = \begin{cases} 
1, & \text{se a pessoa } i \text{ inicia a ocorrência } o \text{ da tarefa } j \text{ no slot } t \\
0, & \text{caso contrário}
\end{cases}$$

Variável auxiliar de balanceamento de carga:

$$\Delta \ge 0 \quad \text{(Representa a maior diferença de percentual de carga entre duas pessoas)}$$

**Função Objetivo**

Minimizar a falta de Aptidão: ANTIGA

$$\min Z = \sum_{i \in \mathcal{P}} \sum_{j \in \mathcal{D}} \sum_{o \in \mathcal{O}_j} \sum_{t \in \mathcal{T}} (1 - c_{i,j}) \cdot x_{i,j,o,t}$$

Minimizar a falta de Aptidão e o Desbalanceamento de Carga:

$$\min Z = \left( \sum_{i \in \mathcal{P}} \sum_{j \in \mathcal{D}} \sum_{o \in \mathcal{O}_j} \sum_{t \in \mathcal{T}} (1 - c_{i,j}) \cdot x_{i,j,o,t} \right) + \mathbf{\alpha \cdot \Delta}$$

**Restrições Principais**

1. Cada ocorrência de tarefa deve ser realizada exatamente uma vez:

$$\sum_{i \in \mathcal{P}} \sum_{t \in \mathcal{T}} x_{i,j,o,t} = 1 \quad \forall j \in \mathcal{D}, \forall o \in \mathcal{O}_j$$

2. Não sobreposição de tarefas por pessoa: Uma pessoa não pode realizar 2 tarefas simultaneamente (sejam elas do bebê ou da casa).

$$\sum_{j \in \mathcal{D}} \sum_{o \in \mathcal{O}_j} \sum_{t' = t}^{t + d_j} x_{i,j,o,t'} \le 1 \quad \forall i \in \mathcal{P}, \forall t \in \mathcal{T}$$

3. Não sobreposição de tarefas do bebê: No máximo uma tarefa do bebê ($\mathcal{D}_{B}$) pode estar ativa em qualquer slot $t$. Vale ressaltar que tarefas da casa ($\mathcal{D}_{C}$) não são limitadas por esta restrição. Isso é necessário pois, por exemplo, não é possível dar banho e amamentar um bebê simultaneamente, porém é possível que uma pessoa lave o banheiro enquanto outra faz comida.

$$\sum_{i \in \mathcal{P}} \sum_{\mathbf{j \in \mathcal{D}_{B}}} \sum_{o \in \mathcal{O}_j} \sum_{t' = \max(0, t - d_j + 1)}^{\min(t, S - d_j)} x_{i,j,o,t'} \le 1 \quad \forall t \in \mathcal{T}$$

4. Respeitar disponibilidade das pessoas: uma pessoa só pode ser alocada se estiver disponível. Vale ressaltar que o modelo não é robusto para o término da tarefa. Então ele checa apenas se a pessoa está disponível para começar a tarefa.

$$x_{i,j,o,t} \le A_{i,t} \quad \forall i \in \mathcal{P}, \forall j \in \mathcal{D}, \forall o \in \mathcal{O}_j, \forall t \in \mathcal{T}$$

5. Respeitar horários das Tarefas: A tarefa $j$ não pode ser iniciada no slot $t$ se o horário estiver bloqueado para ela se $TA_{j,t} = 0$. Vale apenas para inicio inválido, que o vetor $TA_{j,t}$ já define.

$$x_{i,j,o,t} \le TA_{j,t} \quad \forall i \in \mathcal{P}, \forall j \in \mathcal{D}, \forall o \in \mathcal{O}_j, \forall t \in \mathcal{T}$$

6. Precedência entre tarefas (mesma ocorrencia, limitado): Se uma ocorrência da tarefa $j_1$ é iniciada no slot $t_1$ por alguma pessoa, então a ocorrência da tarefa subsequente $j_2$ deve ser iniciada num intervalo permitido ($t_1 + d_{j_1}$ até $t_1 + d_{j_1} + W$, onde $W$ é a janela máxima de espera). Isso garante que tarefas dependentes ocorram na sequência correta e dentro de um intervalo temporal aceitável.

$$x_{i_1,j_1,o,t_1} \le \sum_{i_2 \in \mathcal{P}} \sum_{t_2 = t_1 + d_{j_1}}^{\min(S-d_{j_2}, t_1 + d_{j_1} + W)} x_{i_2,j_2,o,t_2}$$

$$\forall i_1 \in \mathcal{P}, \forall j_1 \to j_2, \forall o \in \mathcal{O}_{j_1}, \forall t_1 \in \mathcal{T}$$

7. Periodicidade (Tarefas Recorrentes): Tarefas que devem ocorrer periodicamente mantêm um espaçamento temporal fixo entre ocorrências consecutivas. Se a ocorrência $o$ de uma tarefa periódica $j$ é iniciada no slot $t_1$, então a ocorrência $o+1$ deve ser iniciada no slot $t_1 + P_j$ (onde $P_j$ é o período). 

$$\sum_{i \in \mathcal{P}} x_{i,j,o,t_1} = \sum_{i' \in \mathcal{P}} x_{i',j,o+1,t_1+P_j}$$

$$\forall j \in \mathcal{D} \text{ com periodicidade } P_j, \quad \forall o \in \{0, \dots, |\mathcal{O}_j| - 2\}, \quad \forall t_1 \in \{0, \dots, S-d_j-P_j\}$$

8. Limite de carga de trabalho: A carga total de trabalho alocada a cada pessoa $i$ não deve exceder seu limite máximo $L_i$. A carga é medida em slots (duração das tarefas), garantindo que ninguém seja sobrecarregado.

$$\sum_{j \in \mathcal{D}} \sum_{o \in \mathcal{O}_j} \sum_{t \in \mathcal{T}} d_j \cdot x_{i,j,o,t} \le L_i \quad \forall i \in \mathcal{P}$$

9. Restrição de Balanceamento Relativo (Soft Constraint): Para garantir que todos trabalhem numa porcentagem similar do seu limite $L_i$, a variável $\Delta$ deve ser maior ou igual à diferença de ocupação entre qualquer par de pessoas $i$ e $k$.

$$\sum_{j,o,t} \frac{d_j}{L_i} x_{i,j,o,t} - \sum_{j,o,t} \frac{d_j}{L_k} x_{k,j,o,t} \le \Delta \quad \forall i, k \in \mathcal{P}, i \neq k$$



### Pré-processamento de dados

In [5]:
import json

# Constantes que NÃO dependem dos dados (dias da semana)
MAPA_DIAS = {
    "seg": 0, "ter": 1, "qua": 2, "qui": 3, "sex": 4, "sab": 5, "dom": 6
}
DIAS_SEMANA = list(MAPA_DIAS.keys())


def _time_para_slot(hora_str, slots_por_dia, duracao_slot):
    """
    (Função interna) Converte 'HH:MM' para um índice de slot.
    """
    if hora_str == "24:00":
        return slots_por_dia
    try:
        horas, minutos = map(int, hora_str.split(':'))
        total_minutos = horas * 60 + minutos
        return total_minutos // duracao_slot
    except ValueError:
        print(f"Erro: Formato de hora inválido '{hora_str}'. Use 'HH:MM'.")
        return 0

def _processar_disponibilidade(regras_disponibilidade, total_slots, slots_por_dia, duracao_slot):
    """
    (Função interna) Converte regras de disponibilidade (pessoas) em vetores binários.
    Entrada:
      - regras_disponibilidade: dicionário com regras por pessoa (campo "availability" do JSON)
      - total_slots: número total de slots no horizonte
      - slots_por_dia: número de slots por dia
      - duracao_slot: duração de cada slot em minutos
    Retorna um dicionário { pessoa: [0/1, ...] } com comprimento total_slots.
    """
    disponibilidade_final = {}

    for pessoa, regras in regras_disponibilidade.items():
        vetor_pessoa = [0] * total_slots

        for regra in regras:
            dias_para_aplicar = []
            if regra["dia"] == "todos":
                dias_para_aplicar = DIAS_SEMANA
            else:
                dias_para_aplicar = [regra["dia"]]

            slot_inicio_dia = _time_para_slot(regra["inicio"], slots_por_dia, duracao_slot)
            slot_fim_dia = _time_para_slot(regra["fim"], slots_por_dia, duracao_slot)

            for nome_dia in dias_para_aplicar:
                if nome_dia not in MAPA_DIAS:
                    print(f"Aviso: Dia '{nome_dia}' inválido para '{pessoa}'. Pulando regra.")
                    continue

                indice_dia = MAPA_DIAS[nome_dia]
                offset_dia = indice_dia * slots_por_dia

                for i in range(slot_inicio_dia, slot_fim_dia):
                    indice_global = offset_dia + i
                    if indice_global < total_slots:
                        vetor_pessoa[indice_global] = 1

        disponibilidade_final[pessoa] = vetor_pessoa

    return disponibilidade_final

def _processar_janelas_tarefas(janelas_tarefas, dados_tarefas, total_slots, slots_por_dia, duracao_slot, total_dias):
    """
    Converte janelas de tempo das tarefas em vetores binários.
    Retorna um dicionário: { "tarefa": [1, 1, 0, 0, ...], ... }
    Parâmetros:
      - janelas_tarefas: dicionário com janelas (campo "horario_tarefas" do JSON)
      - dados_tarefas: dicionário com informações das tarefas (campo "tarefas")
      - total_slots, slots_por_dia, duracao_slot, total_dias: parâmetros de horizonte
    """
    # Inicializa todas as tarefas como 100% disponíveis (vetor de uns)
    disponibilidade_tarefas = {}
    for tarefa in dados_tarefas.keys():
        disponibilidade_tarefas[tarefa] = [1] * total_slots

    # Aplica as restrições definidas em janelas_tarefas
    for nome_tarefa, janelas in janelas_tarefas.items():
        if nome_tarefa not in dados_tarefas:
            print(f"Aviso: Janela definida para tarefa '{nome_tarefa}', mas ela não existe em 'tarefas'.")
            continue

        # Se tem janela definida, começamos zerando a disponibilidade e marcando apenas os inícios permitidos
        disponibilidade_tarefas[nome_tarefa] = [0] * total_slots

        # Quantos slots a tarefa dura
        duracao_em_slots = dados_tarefas[nome_tarefa]["duracao"] // duracao_slot

        for janela in janelas:
            inicio_no_dia = _time_para_slot(janela["inicio"], slots_por_dia, duracao_slot)
            fim_no_dia = _time_para_slot(janela["fim"], slots_por_dia, duracao_slot)

            # Último início válido para que a tarefa caiba na janela
            limite_inicio_valido = fim_no_dia - duracao_em_slots

            # Aplica para cada dia do horizonte (janelas repetidas diariamente)
            for dia in range(total_dias):
                offset = dia * slots_por_dia

                s_min = offset + inicio_no_dia
                s_max = offset + limite_inicio_valido

                # Marca como 1 os slots de início permitidos (note: s_max é exclusivo)
                for t in range(s_min, s_max):
                    if 0 <= t < total_slots:
                        disponibilidade_tarefas[nome_tarefa][t] = 1

    return disponibilidade_tarefas

def carregar_dados(caminho_arquivo):
    """
    Função principal. Carrega JSON, processa Pessoas e Tarefas.
    """
    print(f"Carregando dados de '{caminho_arquivo}'...")
    
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            dados = json.load(f)
    except FileNotFoundError:
        print(f"Erro: Arquivo '{caminho_arquivo}' não encontrado.")
        return None
    except json.JSONDecodeError:
        print(f"Erro: Arquivo '{caminho_arquivo}' não é um JSON válido.")
        return None

    # 2. LÊ AS CONSTANTES
    try:
        duracao_slot = dados['slot_duracao_min']
        total_dias = dados['dias']
        total_slots = total_dias * (24 * 60) // duracao_slot

        regras_disponibilidade = dados["disponibilidade_pessoas"]
        dados_tarefas = dados["tarefas"]
        janelas_tarefas = dados.get("disponibilidade_tarefas", {})

    except KeyError as e:
        print(f"Erro: Chave obrigatória {e} não encontrada no JSON.")
        return None

    # 3. VALIDAÇÕES
    if duracao_slot <= 0:
        print("Erro: 'slot_duration_min' deve ser positivo.")
        return None
        
    slots_por_dia = (24 * 60) // duracao_slot

    # 4. PROCESSA DISPONIBILIDADE DAS PESSOAS
    vetores_pessoas = _processar_disponibilidade(
        regras_disponibilidade,
        total_slots,
        slots_por_dia,
        duracao_slot,
    )
    dados["disponibilidade_pessoas_binaria"] = vetores_pessoas

    # 5. PROCESSA JANELAS DAS TAREFAS 
    print("Processando janelas de tarefas...")
    vetores_tarefas = _processar_janelas_tarefas(
        janelas_tarefas,
        dados_tarefas,
        total_slots,
        slots_por_dia,
        duracao_slot,
        total_dias,
    )
    # Salva no dicionário principal
    dados["disponibilidade_tarefas_binaria"] = vetores_tarefas
    
    print("Dados carregados e processados com sucesso!")
    return dados

## Solução

In [6]:
import json
import pulp
import pandas as pd
import math
import itertools
import time

# Tenta importar tabulate para tabelas bonitas, senão usa pandas padrão
try:
    from tabulate import tabulate
    HAS_TABULATE = True
except ImportError:
    HAS_TABULATE = False

# Script principal para montar e resolver o modelo de alocação de tarefas.
# Este arquivo monta um modelo MILP (pulp) a partir do JSON processado
# por `preprocessamento.carregar_dados(...)` e aplica as restrições

# ==============================
# 1. Leitura e Pré-processamento
# ==============================

# Carrega e processa o JSON com funções utilitárias (converte janelas e disponibilidades
# em vetores binários, calcula slots, etc.). O dicionário retornado contém tanto os dados
# originais quanto as chaves auxiliares `disponibilidade_pessoas_binaria` e `disponibilidade_tarefas_binaria`.
data = carregar_dados("input_semanal_1dia.json")

if data is None:
    print("Erro fatal: Falha ao carregar ou processar os dados. Encerrando.")
    exit()

pessoas = data["pessoas"] # conjunto de pessoas
tarefas = list(data["tarefas"].keys()) # conjunto de tarefas
duracao_slot = data["slot_duracao_min"]
total_slots = data["dias"]*24*(60//duracao_slot) # número total de slots
disponibilidade_tarefas = data.get("disponibilidade_tarefas_binaria") # TA_{j,t}
alpha = data.get("alpha", 0) # Valor padrão 0 se não estiver definido
tarefas_bebe = [j for j, task_data in data["tarefas"].items() if task_data.get("tipo") == "bebe"]
duracao_tarefas = {
    j: math.ceil(data["tarefas"][j]["duracao"] / duracao_slot) 
    for j in tarefas
} # d_j (duração em slots)
ocorrencias = {j: range(data["tarefas"][j]["ocorrencias"]) for j in tarefas}
disponibilidade_pessoas = data["disponibilidade_pessoas_binaria"]
capacidade = data["aptidao"] # c_{i,j}
dependencias =  data["dependencias"] # dependências entre tarefas

limite_carga_horas = data.get("limite_carga_horas", {})
if duracao_slot > 0:
    limite_carga = {i: int((limite_em_horas * 60) / duracao_slot) 
                    for i, limite_em_horas in limite_carga_horas.items()}
else:
    limite_carga = {}
    print("Aviso: 'slot_duracao_min' é 0. O Limite de Carga não será aplicado.")

# ==============================
# 2. Criação do modelo
# ==============================
print ("ALPHA: ", alpha )
model = pulp.LpProblem("x_Cuidados_Bebe", pulp.LpMinimize)

# Variável Delta para o Balanceamento 
# Representa a maior diferença de % de carga entre duas pessoas
delta_balanceamento = pulp.LpVariable("Delta_Balanceamento", lowBound=0, cat="Continuous")

# Variáveis de decisão: x[i][j][o][t] = 1 se pessoa i inicia tarefa j, ocorrência o no slot t
x = {}
for i in pessoas:
    x[i] = {}
    for j in tarefas:
        x[i][j] = {}
        for o in ocorrencias[j]:
            x[i][j][o] = {}
            for t in range(total_slots):
                x[i][j][o][t] = pulp.LpVariable(f"x_{i}_{j}_{o}_{t}", cat="Binary")

# Proibir inícios inválidos: se t + D[j] > Slots então x[i][j][o][t] == 0
for i in pessoas:
    for j in tarefas:
        dur = duracao_tarefas[j]
        ultimo_inicio = total_slots - dur
        for o in ocorrencias[j]:
            for t in range(total_slots):
                if t > ultimo_inicio:
                    model += x[i][j][o][t] == 0

# ==============================
# 3. Função Objetivo
# ==============================

# Termo 1: Aptidão (Minimizar falta de aptidão)
objetivo_aptidao = pulp.lpSum(
    (1 - capacidade[i][j]) * x[i][j][o][t]
    for i in pessoas
    for j in tarefas 
    for o in ocorrencias[j]
    for t in range(total_slots)
)

# Termo 2: Penalidade de Desequilíbrio (alpha * delta)
model += objetivo_aptidao + (alpha * delta_balanceamento)

# ==============================
# 4. Restrições
# ==============================

# 4.1 Cada ocorrência de tarefa deve ser realizada exatamente uma vez
for j in tarefas:
    for o in ocorrencias[j]:
        model += pulp.lpSum(x[i][j][o][t] for i in pessoas for t in range(total_slots)) == 1

# 4.2 Não sobreposição de tarefas por pessoa
for i in pessoas:
    for t in range(total_slots):
        model += (
            pulp.lpSum(
                x[i][j][o][t_start]
                for j in tarefas
                for o in ocorrencias[j]
                for t_start in range(max(0, t - duracao_tarefas[j] + 1), t + 1)
            )
            <= 1
        )

# 4.3 Não sobreposição de tarefas do bebê
for t in range(total_slots):
    expr = []
    for i in pessoas:
        for j in tarefas_bebe:
            dur = duracao_tarefas[j]
            t_start_min = max(0, t - dur + 1)
            t_start_max = min(t, total_slots - dur)
            for o in ocorrencias[j]:
                for t_start in range(t_start_min, t_start_max + 1):
                    expr.append(x[i][j][o][t_start])
    model += pulp.lpSum(expr) <= 1


# 4.4 Respeitar disponibilidade das pessoas
for i in pessoas:
    for j in tarefas:
        for o in ocorrencias[j]:
            for t in range(total_slots):
                if disponibilidade_pessoas[i][t] == 0:
                    model += x[i][j][o][t] == 0

# 4.5 Respeitar horários das Tarefas
# Se disponibilidade_tarefas[j][t] == 0, a tarefa j não pode começar no slot t.
for j in tarefas:
    if disponibilidade_tarefas and 0 in disponibilidade_tarefas[j]:
        for o in ocorrencias[j]:
            for t in range(total_slots):
                if disponibilidade_tarefas[j][t] == 0:
                    for i in pessoas:
                        model += x[i][j][o][t] == 0

# 4.6 Precedência entre tarefas
for j1_id in dependencias:
    j2_id = dependencias[j1_id]["proxima_tarefa"]
    W = math.ceil(dependencias[j1_id]["janela_de_espera"] / duracao_slot)
    print(f"Aplicando precedência: {j1_id} -> {j2_id} com janela {W} slots.")

    for o in ocorrencias[j1_id]:
        for t2 in range(total_slots):
            d_j1 = duracao_tarefas[j1_id]
            t1_min = max(0, t2 - d_j1 - W)
            t1_max = min(t2 - d_j1, total_slots - d_j1)

            if t1_min > t1_max:
                # Se não existe nenhum início válido de j1 que permita j2 iniciar em t2,
                # então nenhum i2 pode iniciar j2 em t2 (forçamos a variável a 0).
                for i2 in pessoas:
                    model += x[i2][j2_id][o][t2] == 0
                continue

            lhs = []
            for i1 in pessoas:
                for t1 in range(t1_min, t1_max + 1):
                    lhs.append(x[i1][j1_id][o][t1])

            for i2 in pessoas:
                model += pulp.lpSum(lhs) >= x[i2][j2_id][o][t2]

# 4.6 Restrição de periodicidade (tarefas recorrentes)
if "periodicidade" in data:
    for j, P_j in data["periodicidade"].items():
        P_j_slots = math.ceil(P_j / duracao_slot)
        dur = duracao_tarefas[j]
        last_start = total_slots - dur
        for o in range(len(ocorrencias[j]) - 1):
            for t1 in range(0, last_start + 1):
                t2 = t1 + P_j_slots
                if t2 <= last_start:
                    model += pulp.lpSum(x[i][j][o][t1] for i in pessoas) == \
                             pulp.lpSum(x[i][j][o+1][t2] for i in pessoas)
                else:
                    # Se o deslocamento pela periodicidade ultrapassa o horizonte,
                    # então um início em t1 não é válido (eliminação de inicios inválidos).
                    model += pulp.lpSum(x[i][j][o][t1] for i in pessoas) == 0

# 4.8 e 4.9 : Limites e Balanceamento

# 1. Pré-cálculo das Expressões de Carga
# Isso cria a expressão linear da carga total (em slots) para cada pessoa UMA ÚNICA VEZ.
expressao_carga_pessoa = {}

for i in pessoas:
    # Monta a soma de (duração * variável_decisao)
    expressao_carga_pessoa[i] = pulp.lpSum(
        duracao_tarefas[j] * x[i][j][o][t]
        for j in tarefas
        for o in ocorrencias[j]
        for t in range(total_slots)
    )

# 2. Aplicação da Restrição "Hard" (Limite Máximo)
# Ninguém pode ultrapassar seu teto de horas, independente do balanceamento.
if limite_carga:
    for i in pessoas:
        if i in limite_carga:
            L_i = limite_carga[i]
            # Usa a expressão pré-calculada (muito mais rápido)
            model += expressao_carga_pessoa[i] <= L_i, f"Limite_Maximo_{i}"

# 3. Aplicação da Restrição "Soft" (Balanceamento Relativo / Minimax)
# Tenta igualar a % de ocupação entre as pessoas.
if limite_carga and alpha > 0:
    # Filtra apenas pessoas com limite definido > 0
    pessoas_validas = [p for p in pessoas if p in limite_carga and limite_carga[p] > 0]
    
    if len(pessoas_validas) >= 2:
        # itertools.combinations evita pares duplicados e auto-comparação (ex: A-B é igual B-A)
        for p1, p2 in itertools.combinations(pessoas_validas, 2):
            L1 = float(limite_carga[p1])
            L2 = float(limite_carga[p2])
            
            # Percentual de uso = Carga Real / Limite Total
            pct_p1 = expressao_carga_pessoa[p1] / L1
            pct_p2 = expressao_carga_pessoa[p2] / L2
            
            # Para forçar Delta >= |pc (norma infinita)t_p1 - pct_p2|, adicionamos duas restrições lineares:
            
            # 1. (P1 - P2) <= Delta
            model += pct_p1 - pct_p2 <= delta_balanceamento, f"Balanceamento_{p1}_{p2}_pos"
            
            # 2. (P2 - P1) <= Delta
            model += pct_p2 - pct_p1 <= delta_balanceamento, f"Balanceamento_{p1}_{p2}_neg"


# ======================================
# 5. Resolver modelo e mostrar solução
# ======================================

solver = pulp.getSolver('HiGHS', timeLimit=300, msg=False)


model.solve(solver)
#model.solve(pulp.PULP_CBC_CMD(msg=False))

status_code = model.status
status_string = pulp.LpStatus[status_code]
print(f"Status do Modelo: {status_string}")

if status_string != "Optimal" and status_string != "Feasible": # HiGHS pode retornar Feasible com Gap
    print("\nO modelo NÃO ENCONTROU uma solução viável.")
else:
    print(f"Valor da Função Objetivo: {pulp.value(model.objective):.2f}")


# ==============================
# 6. Exportar solução em JSON
# ==============================

if status_string == "Optimal" or status_string == "Feasible":
    solution = []
    for i in pessoas:
        for j in tarefas:
            for o in ocorrencias[j]:
                for t in range(total_slots):
                    if pulp.value(x[i][j][o][t]) > 0.99:
                        # Cálculos de tempo
                        slots_por_dia = (24 * 60) // duracao_slot
                        inicio_slot = t
                        fim_slot = t + duracao_tarefas[j]

                        dia_inicio = (inicio_slot // slots_por_dia) + 1
                        slot_no_dia_inicio = inicio_slot % slots_por_dia
                        minutos_inicio = slot_no_dia_inicio * duracao_slot
                        hora_inicio = f"{minutos_inicio // 60:02d}:{minutos_inicio % 60:02d}"

                        dia_fim = (fim_slot // slots_por_dia) + 1
                        slot_no_dia_fim = fim_slot % slots_por_dia
                        minutos_fim = slot_no_dia_fim * duracao_slot
                        hora_fim = f"{minutos_fim // 60:02d}:{minutos_fim % 60:02d}"

                        solution.append({
                            "dia_inicio": dia_inicio,
                            "hora_inicio": hora_inicio,
                            "hora_fim": hora_fim,
                            "pessoa": i,
                            "tarefa": j,
                            "ocorrencia": o,
                            "inicio_slot": inicio_slot, # mantido para ordenação
                            "fim_slot": fim_slot
                        })

    # Ordena por tempo global
    solution = sorted(solution, key=lambda x: x["inicio_slot"])
    
    # Cria DataFrame
    df = pd.DataFrame(solution)

    # ---------------------------------------------------------
    # IMPRESSÃO TABULAR NO TERMINAL (SEPARADA POR DIA)
    # ---------------------------------------------------------
    colunas_visuais = ['hora_inicio', 'hora_fim', 'pessoa', 'tarefa', 'ocorrencia']
    
    print("\n" + "="*50)
    print("           CRONOGRAMA DETALHADO")
    print("="*50)

    dias_unicos = sorted(df['dia_inicio'].unique())

    for dia in dias_unicos:
        print(f"\n>>> DIA {dia}")
        df_dia = df[df['dia_inicio'] == dia][colunas_visuais]
        
        if HAS_TABULATE:
            # Opções de tablefmt: 'psql', 'grid', 'simple', 'github'
            print(tabulate(df_dia, headers='keys', tablefmt='psql', showindex=False))
        else:
            print(df_dia.to_string(index=False))


else:
    print("\nNenhuma solução viável foi encontrada para exportar.")



Carregando dados de 'input_semanal_1dia.json'...
Processando janelas de tarefas...
Dados carregados e processados com sucesso!
ALPHA:  1
Aplicando precedência: amamentar -> arrotar com janela 0 slots.
Aplicando precedência: preparar_cafe_manha -> lavar_louca_cafe com janela 10 slots.
Aplicando precedência: preparar_almoco -> lavar_louca_almoco com janela 10 slots.
Aplicando precedência: preparar_jantar -> lavar_louca_jantar com janela 10 slots.
Aplicando precedência: amamentar -> arrotar com janela 0 slots.
Aplicando precedência: preparar_cafe_manha -> lavar_louca_cafe com janela 10 slots.
Aplicando precedência: preparar_almoco -> lavar_louca_almoco com janela 10 slots.
Aplicando precedência: preparar_jantar -> lavar_louca_jantar com janela 10 slots.
Aplicando precedência: lavar_roupa -> estender_roupa com janela 2 slots.
Aplicando precedência: lavar_roupa -> estender_roupa com janela 2 slots.
Status do Modelo: Optimal
Valor da Função Objetivo: 6.68

           CRONOGRAMA DETALHADO

>>

## Discussão

A modelagem inicial não considerava o horário das tarefas, o que acarretou em tarefas sendo feitas em horários não funcionais, por exemplo, café da manhã sendo feito a tarde. Dessa forma, surgiu a necessidade de incluir essa restrição. A restrição de horário é um restrição forte e, ao colocá-la, trouxe dificuldades no que diz respeito a satisfabilidade do problema. Algumas vezes o solver não encontrou solução viável. A justificativa encontrada na análise é que conciliar o horário das tarefas com algumas precedências é muito restritivo para o problema. 

Além disso, outra restrição que não estava prevista inicialmente era a restrição soft de equilíbrio de carga. Essa restrição afeta a função objetivo, pois a tendência vai ser equilibrar a diferença entre as cargas alocadas. A ideia é que um alpha maior dê mais peso para essa homogeneidade da carga, enquanto um alpha menor, dá mais peso para a adequação pela aptidão das pessoas. 

Foram utilizados 2 solvers no teste, o CBC e o HiGHs, ambos utilizados pela biblioteca PuLP do python. Como o conjunto de dados para teste gerava uma grande quantidade de variáveis, o CBC não soube lidar quando o alpha aumentava e travava para encontrar a solução ótima, ficando em loop no mesmo valor de solução, sem melhorar e sem achar o ótimo, uma possibilidade para isso pode ser o jeito que o solver implementa os cortes. Já o HiGHs se mostrou melhor no que diz respeito ao desempenho de diferentes valores de alpha, suportando diferentes valores. Dessa forma, foi decidido que o solver a ser utilizado e considerado os resultados seria o HiGHs.

Para testar o problema, já completo com as restrições finais, foi utilizado um conjunto de dados fictícios definidos em um json com a seguinte forma:

1. Configuração geral
 - Parâmetro Alpha: inteiro, variável (foram testados diferentes valores)
 - Período: 1 dia (a escolha de 1 dia foi feita pois o problema gera muitas variáveis)
 - Duração do Slot de Tempo: 30 minutos (foram testados slots de 15 e 30 minutos, sendo o de 30 menor e mais rápido)

2. Pessoas na rede de apoio: mãe, pai, avó, tia, tio e babá

3. Tarefas 

<div align="center">

| Tarefa | Duração (min) | Ocorrências | Categoria |
| :--- | :---: | :---: | :---: |
| Amamentar | 60 | 8 | Bebê |
| Arrotar | 30 | 8 | Bebê |
| Trocar Fralda | 30 | 8 | Bebê |
| Ninar | 30 | 8 | Bebê |
| Banho | 30 | 1 | Bebê |
| Cortar Unha | 15 | 1 | Bebê |
| Banho de Sol | 15 | 1 | Bebê |
| Lavar Louça do Café | 15 | 1 | Casa |
| Lavar Louça do Almoço | 15 | 1 | Casa |
| Lavar Louça do Jantar | 15 | 1 | Casa |
| Lavar Banheiro | 30 | 1 | Casa |
| Tirar Lixo | 15 | 1 | Casa |
| Aspirar Casa | 30 | 1 | Casa |
| Lavar Roupa | 15 | 1 | Casa |
| Passar Roupa | 30 | 1 | Casa |
| Estender Roupa | 15 | 1 | Casa |
| Preparar Café da Manhã | 15 | 1 | Casa |
| Preparar Jantar | 30 | 1 | Casa |
| Preparar Almoço | 60 | 1 | Casa |

</div>

4. Disponibilidade de horários (todos em dia pois esses dados são apenas de 1 dia, quando tem mais, colocá-se os dias da semana: "seg", "ter, ... , "dom")

<div align="center">

| Pessoa | Dia | Início | Fim |
| :---: | :---: | :---: | :---: |
| **mae** | todos | 00:00 | 24:00 |
| **pai** | todos | 00:00 | 08:00 |
| **pai** | todos | 17:00 | 24:00 |
| **avo** | todos | 07:00 | 16:00 |
| **tio** | todos | 15:00 | 21:00 |
| **tia** | todos | 15:00 | 17:00 |
| **baba** | todos | 10:00 | 15:00 |

</div>

5. Matriz de Capacidade (Eficiência)

Valores indicam a aptidão da pessoa para realizar a tarefa (0.0 a 1.0). Vale ressaltar que a babá possui 0.0 nas tarefas de casa pois ela não deveria ser alocada para tais. 

<div align="center">

|                     |   Mae |   Pai |   Avo |   Tio |   Tia |   Babá |
|:--------------------|:-----:|:-----:|:-----:|:---:|:---:|:-------:|
| amamentar           |   1   |   0.5 |   0.5 |   0.5 |   0.5 |    0.5 |
| arrotar             |   0.9 |   0.9 |   0.9 |   0.9 |   0.9 |    0.9 |
| trocar_fralda       |   0.8 |   0.7 |   0.7 |   0.3 |   0.4 |    0.9 |
| ninar               |   0.8 |   0.8 |   0.8 |   0.8 |   0.8 |    0.9 |
| banho               |   0.9 |   0.9 |   0.9 |   0.9 |   0.9 |    0.9 |
| cortar_unha         |   0.8 |   0.6 |   0.9 |   0.4 |   0.7 |    0.8 |
| banho_de_sol        |   1   |   1   |   1   |   1   |   1   |    1   |
| lavar_louca_cafe    |   0.7 |   0.8 |   0.6 |   0.5 |   0.7 |    0   |
| lavar_louca_almoco  |   0.7 |   0.7 |   0.7 |   0.7 |   0.7 |    0   |
| lavar_louca_jantar  |   0.7 |   0.7 |   0.7 |   0.7 |   0.7 |    0   |
| lavar_banheiro      |   0.3 |   0.8 |   0.4 |   0.6 |   0.5 |    0   |
| tirar_lixo          |   0.6 |   0.9 |   0.5 |   0.6 |   0.6 |    0   |
| aspirar_casa        |   0.5 |   0.8 |   0.4 |   0.5 |   0.6 |    0   |
| lavar_roupa         |   0.8 |   0.8 |   0.5 |   0.4 |   0.6 |    0   |
| passar_roupa        |   0.7 |   0.6 |   0.5 |   0.3 |   0.5 |    0   |
| estender_roupa      |   0.9 |   0.9 |   0.5 |   0.4 |   0.7 |    0   |
| preparar_cafe_manha |   0.8 |   0.8 |   0.7 |   0.6 |   0.7 |    0   |
| preparar_jantar     |   0.8 |   0.8 |   0.8 |   0.5 |   0.7 |    0   |
| preparar_almoco     |   0.8 |   0.8 |   0.9 |   0.5 |   0.7 |    0   |

</div>

6. Dependências de Tarefas

<div align="center">

| Tarefa Atual | Próxima Tarefa | Janela (min) |
| :---: | :---: | :---: |
| amamentar | arrotar | 0 |
| preparar_cafe_manha | lavar_louca_cafe | 300 |
| preparar_almoco | lavar_louca_almoco | 300 |
| preparar_jantar | lavar_louca_jantar | 300 |
| lavar_roupa | estender_roupa | 60 |

</div>

7. Periodicidade (Repetição)
Tarefas que devem ocorrer a cada X minutos.

<div align="center">

| Tarefa | Intervalo (min) |
| :--- | :---: |
| ninar | 180 |
| amamentar | 180 |
| trocar_fralda | 180 |

</div>

8. Limite de Carga de Trabalho
Máximo de horas trabalhadas por dia.

<div align="center">

| Pessoa | Limite (horas) |
| :---: | :---: |
| mae | 16 |
| pai | 8 |
| avo | 5 |
| tio | 5 |
| tia | 5 |
| Babá | 6 |

</div>

9. Janelas de Execução
Intervalos de horário permitidos para tarefas específicas.

<div align="center">

| Tarefa | Início | Fim |
| :--- | :---: | :---: |
| banho | 07:00 | 21:00 |
| banho_de_sol | 07:00 | 10:00 |
| banho_de_sol | 15:00 | 17:00 |
| preparar_cafe_manha | 05:00 | 09:00 |
| preparar_almoco | 09:00 | 13:00 |
| preparar_jantar | 17:00 | 22:00 |

</div>

Como dito anteriormente, foram testados alguns valores de alpha, como 0, 1, 10, 100 e 1000. Foi possível observar a diferença entre como as tarefas são alocadas e o valor da função objetivo. 

<div align="center">

| alpha | tempo | valor da função objetivo | iterações LP |
| :---: | :---: | :---: | :---: |
| 0 | 1.67s | 6.2 | 412  |
| 0.01 | 13.94s | 6.205 | 618 |
| 1 | 13.06s | 6.68 | 1303 |
| 10 | 11.37 | 9.175 | 918 |
| 100 | 16.64 | 24.467 | 4460 |
| 1000 | 10.80 | 174.46 | 1795 |


</div>

### Limitações 

A modelagem do problema tem algumas limitações, sendo elas: 

- A disponibilidade da pessoa executar a tarefa considera apenas o começo. Dessa forma, se alguém está disponível de 8h às 10h, o modelo considera que ela pode começar uma tarefa até 10h. Sendo assim, pode alocar, por exemplo, de 9h30 as 10h30. 
- A precedência entre tarefas considera apenas a mesma ocorrência de tarefas diferentes. Por exemplo, se existe a precedência entre a tarefa x e y, significa que cada uma das ocorrências de y ocorre depois da ocorrência de mesmo número da tarefa x. Essa limitação foi percebida na insatisfabilidade quando lavar Louça deveria acontecer depois do café da manhã, almoço e jantar. Essa insatisfabilidade faz total sentido, pois o modelo estava tentando alocar a ocorrência 0 de lavar a louça após as três tarefas respeitando uma janela, o que não era possível. Para mitigar esse problema, ao invés de 3 instâncias da tarefa lavar louça, foram colocadas 3 tarefas diferentes para lavar a louça em cada momento, então a precedência ocorre corretamente. 

### Trabalhos Futuros 

Os trabalhos futuros incluem principalmente o desenvolvimento de uma modelagem mais robusta às limitações apresentadas. Além disso, um ponto a ser considerado é o solver utilizado. Pois o solver Open-source HiGHs é mais lendo do que outros como Gurobi e CPLEX, o que pode tornar a resolução mais escalável. 